### RNN 활용 - 단어 자동 완성하기 ¶
4개의 글자를 가진 단어를 학습시켜, 3글자만 주어지면 나머지 한글자를 추천하여 단어를 완성

In [0]:
import tensorflow as tf
import numpy as np

import os, warnings
warnings.filterwarnings(action="ignore")

import time

**알파벳 리스트 지정**

In [0]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g','h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u','v', 'w', 'x', 'y', 'z']

In [5]:
#one-hot encodiing

num_dic={n:i for i,n in enumerate(char_arr)}
dic_len=len(num_dic)
print(dic_len)

26


In [0]:
#test data

seq_data = ['word', 'wood', 'deep', 'dive','cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [7]:
# 각 단어의 한글자 한글자 index를 저장
print([num_dic[n] for n in seq_data[0][:-1]])
print(np.eye(dic_len)[[22, 14, 17]]) #w:22 #o:14 #r:17

[22, 14, 17]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]]


In [0]:
def make_batch(seq_data, dic_len_num):
  input_batch = []
  target_batch = []
  for seq in seq_data:
# input은 단어의 글자 알파벳 배열의 인덱스 번호 입니다.
# [22, 14, 17] [22, 14, 14] [3, 4, 4] [3, 8, 21] ...
    input = [num_dic[n] for n in seq[:-1]]
    target = num_dic[seq[-1]] # 마지막 글자 인덱스
# 3글자에 대한 one-hot 인코딩을 합니다.
# if input is [0, 1, 2]:
# [[ 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
# [ 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
# [ 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
    input_batch.append(np.eye(dic_len_num)[input])
    target_batch.append(target)
  return input_batch, target_batch

지금까지 손실함수로 사용하던 

softmax_cross_entropy_with_logits 함수는

label 값을 one-hot 인코딩으로 넘겨줘야 하지만,

이 예제에서 사용할 손실 함수인

sparse_softmax_cross_entropy_with_logits 는

one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨주면 됩니다.

In [9]:
# sth in test data

seq_data_test = ['work', 'book']
a, b= make_batch(seq_data_test, dic_len) 
print("a의 값 : 3글자의 알파벳 인덱스 원핫값") # w, o, r
print("전체 단어수 : {}".format(len(a)))
print("첫번째 단어 3글자 :")
print(a[0]) # w, o, r
print("두번째 단어 3글자 :")
print(a[1]) # b, o, o
print("b의 값 : 마지막 글자 index") # k, k
print(b) # d
print("a와 b의 리스트 길이") # k, k
print(len(a), len(b)) # d

a의 값 : 3글자의 알파벳 인덱스 원핫값
전체 단어수 : 2
첫번째 단어 3글자 :
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
  0. 0.]]
두번째 단어 3글자 :
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0.]]
b의 값 : 마지막 글자 index
[10, 10]
a와 b의 리스트 길이
2 2


### 파라미터 설정
learning_rate = 0.01 (학습률)

n_hidden = 128 (은닉층 노드 개수)

total_epoch = 30 (전체 학습 epoch 수)

n_step = 3 (RNN구성 Step 수)

입력값 크기, 알파벳 one-hot 인코딩 개수
* n_input = n_class = dic_len
* 예) c => [0 0 1 0 0 0 0 0 0 0 0 ... 0]
* 출력값도 입력값과 마찬가지로 26개의 알파벳으로 분류.





In [0]:
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
n_step = 3
n_input = n_class = dic_len

**Build the Model**

In [0]:
X = tf.placeholder(tf.float32, [None, n_step, n_input]) # 입력
Y = tf.placeholder(tf.int32, [None]) # 출력
W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

DropoutWrapper 함수 : RNN에도 과적합 방지를 위한 DropoutWrapper 함수 적용이 가능하다.

In [12]:
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 첫번째 셀 생성
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5) # Dropout

cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden) # 두번째 셀 생성

# 에러 발생시 커널 restart
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2]) # 멀티 셀 생성.

outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32) # RNN신경망

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
print("multi_cell ======>")
print(multi_cell)
print()
print("outputs ======>")
print(outputs)
print()
print("states ======>")
print(states)

multi_cell ======>

outputs ======>
Tensor("rnn/transpose_1:0", shape=(?, 3, 128), dtype=float32)

states ======>
(LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(?, 128) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_5:0' shape=(?, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_6:0' shape=(?, 128) dtype=float32>))


RNN 신경망이 출력값의 형태 -> [batch_size, n_step, n_hidden]

이를 실측값과 비교를 위해 아래와 같이 변경
* (가) 순서 바꾸기 :
[batch_size, n_step, n_hidden] -> [n_step, batch_size, n_hidden]
* (나) n_step의 차원을 제거 [batch_size, n_hidden]

In [0]:
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

In [0]:
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
input_batch, target_batch = make_batch(seq_data, dic_len)

In [18]:
for epoch in range(total_epoch):
  _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})
  print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 4.038974
Epoch: 0002 cost = 2.932127
Epoch: 0003 cost = 1.651059
Epoch: 0004 cost = 1.540198
Epoch: 0005 cost = 1.235522
Epoch: 0006 cost = 0.451804
Epoch: 0007 cost = 0.735803
Epoch: 0008 cost = 0.639999
Epoch: 0009 cost = 0.347910
Epoch: 0010 cost = 0.137001
Epoch: 0011 cost = 0.237995
Epoch: 0012 cost = 0.139959
Epoch: 0013 cost = 0.263118
Epoch: 0014 cost = 0.159431
Epoch: 0015 cost = 0.064070
Epoch: 0016 cost = 0.088426
Epoch: 0017 cost = 0.322200
Epoch: 0018 cost = 0.079338
Epoch: 0019 cost = 0.060796
Epoch: 0020 cost = 0.144345
Epoch: 0021 cost = 0.122986
Epoch: 0022 cost = 0.090734
Epoch: 0023 cost = 0.061150
Epoch: 0024 cost = 0.319478
Epoch: 0025 cost = 0.077612
Epoch: 0026 cost = 0.203435
Epoch: 0027 cost = 0.072221
Epoch: 0028 cost = 0.040735
Epoch: 0029 cost = 0.004917
Epoch: 0030 cost = 0.029951
최적화 완료!


**Predict**

In [19]:
# 레이블값이 정수이므로 예측값도 정수로 변경해줍니다.
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
print("prediction : ", prediction)
# one-hot 인코딩이 아니므로 입력값을 그대로 비교합니다.
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))
print("accuracy : ", accuracy)

prediction :  Tensor("Cast:0", shape=(?,), dtype=int32)
accuracy :  Tensor("Mean_1:0", shape=(), dtype=float32)


In [0]:
seq_data_test = ['hard', 'okay', 'deep', 'live','call', 'cool', 'load', 'love', 'kiss', 'kind']

In [21]:
input_batch, target_batch = make_batch(seq_data_test, dic_len)

predict, accuracy_val = sess.run([prediction, accuracy],feed_dict={X: input_batch, Y: target_batch})
print(predict, accuracy_val)

[ 3  3 15  4  3 11  3  4 18  3] 0.8


In [22]:
predict_words = []
for idx, val in enumerate(seq_data_test):
  last_char = char_arr[predict[idx]]
  predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data_test])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['har ', 'oka ', 'dee ', 'liv ', 'cal ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['hard', 'okad', 'deep', 'live', 'cald', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 0.8
